In [124]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException


In [125]:
import pandas as pd

In [126]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
url = "https://www.amazon.es/"
time.sleep(3)

In [127]:
driver.get(url)

In [137]:
texto_a_buscar = 'Tarjeta Grafica'

In [128]:
time.sleep(3)

try:
    busqueda = driver.find_element(By.CSS_SELECTOR,'input#twotabsearchtextbox')

    busqueda.clear()
    busqueda.send_keys(texto_a_buscar)
    busqueda.send_keys(Keys.ENTER)

except NoSuchElementException:

    busqueda = driver.find_element(By.CSS_SELECTOR,'input#nav-bb-search')

    busqueda.clear()
    busqueda.send_keys(texto_a_buscar)
    busqueda.send_keys(Keys.ENTER)

In [129]:
nombre= []
precios = []
caracteristicas = []
stars = []
ratings = []

div.a-section.a-spacing-base

In [130]:
time.sleep(1)
elements = driver.find_elements(By.CSS_SELECTOR,'div.a-section.a-spacing-base')
   
for element in elements:
   
    btn = element.find_element(By.TAG_NAME,"a")

    enlace_url = btn.get_attribute('href')

    driver.execute_script("window.open('" + enlace_url + "', '_blank');")

    # Manejar la nueva ventana
    nuevas_ventanas = driver.window_handles



    ####
    ####


    # -----------
    #Titulo

    title = driver.find_element(By.CSS_SELECTOR,"h1#title").text
    nombre.append(title)

    # ------------
    #Precio


    dollars = driver.find_element(By.CSS_SELECTOR, 'span.a-price-whole').text

    dollars_format = dollars.replace('\n', ' ')
    precios.append(dollars_format)



    # -----------
    # Caracteristicas


    driver.switch_to.window(nuevas_ventanas[-1])  # Cambiar al último manejador de ventana

    try:

        charss = driver.find_element(By.CSS_SELECTOR, 'table#productDetails_techSpec_section_1')


        # Crear un diccionario para almacenar las etiquetas y valores
        info_dict = {}



        # Iterar sobre las filas de la tabla
        for fila in charss.find_elements(By.TAG_NAME, 'tr'):

            try:
            # Obtener la etiqueta y el valor
            
                etiqueta = fila.find_element(By.CLASS_NAME, 'a-size-base.prodDetSectionEntry').text.strip()
                valor = fila.find_element(By.CLASS_NAME, 'a-size-base.prodDetAttrValue').text.strip()

                # Agregar la información al diccionario
                info_dict[etiqueta] = valor

            except NoSuchElementException:
                continue


        caracteristicas.append(info_dict)

    except NoSuchElementException:

        caracteristicas.append(None)


    # Cerramos la ventana
    driver.close()
    driver.switch_to.window(nuevas_ventanas[0])


    #----------------
    # Stars and rating

    try:
        stars_and_ratings = driver.find_element(By.CSS_SELECTOR, 'div#averageCustomerReviews').text.split('\n')

        starsss = stars_and_ratings[0]
        ratingsss = stars_and_ratings[-1]

        stars.append(starsss)

        ratings.append(ratingsss)

    except NoSuchElementException:

        stars.append(None)

        ratings.append(None)


    ####
    ####


    



print(nombre)
print(precios)
print(caracteristicas)
print(stars)
print(ratings)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div#averageCustomerReviews"}
  (Session info: chrome=119.0.6045.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55bdb09745e3 <unknown>
#1 0x55bdb06370b7 <unknown>
#2 0x55bdb0684f53 <unknown>
#3 0x55bdb0685051 <unknown>
#4 0x55bdb06ca9c4 <unknown>
#5 0x55bdb06abf1d <unknown>
#6 0x55bdb06c7b3d <unknown>
#7 0x55bdb06abcc3 <unknown>
#8 0x55bdb06770e4 <unknown>
#9 0x55bdb06780ae <unknown>
#10 0x55bdb093ace1 <unknown>
#11 0x55bdb093eb7e <unknown>
#12 0x55bdb09284b5 <unknown>
#13 0x55bdb093f7d6 <unknown>
#14 0x55bdb090bdbf <unknown>
#15 0x55bdb0962748 <unknown>
#16 0x55bdb0962917 <unknown>
#17 0x55bdb0973773 <unknown>
#18 0x7f6bbbc8f6ba <unknown>


In [ ]:
print(nombre)
print(precios)
print(caracteristicas)
print(stars)
print(ratings)

['Amazon Basics Cable HDMI 2.0 de alta velocidad Ultra HD, compatible con formatos 3D y con canal de retorno de audio, 1.8 m, 6 Pies, Negro', 'Snowkids Cable HDMI 8K@60HZ, 2metros Cable HDMI 2.1 trenzado Alta velocidad 48Gbps 4K@120Hz 7680P Dolby Vision, DTS: X, HDCP 2.2 y 2.3, HDR 10, eARC, Dynamic HDR, Proyector compatible con PS3/4', 'SHULIANCABLE Cable HDMI 4K, HDMI 2.0 Alta Velocidad con Ethernet 4K 18Gbps, 3D, 1080p, Compatible con TV, Xbox, HDR, ARC, PS4, BLU-Ray (1M)', 'Twozoh Cable HDMI 4K 2M, Cable HDMI 2.0 de Alta Velocidad 18Gbps, Cable HDMI Trenzado Compatible con PS5, PS3, PS4, PC, Proyector, HDTV, Xbox', 'Amazon Basics Cable HDMI 2.0 de alta velocidad Ultra HD, compatible con formatos 3D y con canal de retorno de audio, 1.8 m, 6 Pies, Negro', 'conecto Cable HDMI Alta Velocidad con Ethernet, Ultra-HD, conector chapado en oro, 4K, negro, 1,50m', 'KabelDirekt – Cable HDMI 8K / 4K con blindaje A.I.S. – 3m (4K@120Hz/8K@60Hz para una espectacular experiencia Ultra HD – High Sp

In [136]:
productos_amazon={
    'Nombre': nombre,
    'precio':precios,
    'caracteristicas': caracteristicas,
    'stars': stars,
    'ratings': ratings
}

df = pd.DataFrame(productos_amazon)
df

,Nombre,precio,caracteristicas,stars,ratings
0,Amazon Basics Cable HDMI 2.0 de alta velocidad...,7 04€,"{'Marca': 'Amazon Basics', 'Fabricante': 'Amaz...","4,6",541.260 valoraciones
1,"Snowkids Cable HDMI 8K@60HZ, 2metros Cable HDM...",11 99€,"{'Marca': 'Snowkids', 'Fabricante': 'Snowkids'...","4,7",5.474 valoraciones
2,"SHULIANCABLE Cable HDMI 4K, HDMI 2.0 Alta Velo...",Traducir todas las opiniones al español,"{'Marca': 'SHULIANCABLE', 'Fabricante': 'SHULI...","4,6",1.060 valoraciones
3,"Twozoh Cable HDMI 4K 2M, Cable HDMI 2.0 de Alt...",,"{'Marca': 'Twozoh', 'Fabricante': 'Zhonghua', ...","4,5",3.770 valoraciones
4,Amazon Basics Cable HDMI 2.0 de alta velocidad...,7 04€,"{'Marca': 'Amazon Basics', 'Fabricante': 'Amaz...","4,6",541.260 valoraciones
5,conecto Cable HDMI Alta Velocidad con Ethernet...,3 99€,"{'Marca': 'conecto', 'Fabricante': 'SatChef Gm...","4,4",25 valoraciones
6,KabelDirekt – Cable HDMI 8K / 4K con blindaje ...,8 39€,no tiene tabla,"4,6",153.055 valoraciones
7,"JSAUX Cable HDMI 3 Metros 8K 2 Paquetes, Cable...",,no tiene tabla,"4,7",9.517 valoraciones
8,"Snowkids Cable HDMI 8K@60HZ, 2metros Cable HDM...",11 99€,"{'Marca': 'Snowkids', 'Fabricante': 'Snowkids'...","4,7",5.474 valoraciones
9,"FeizLink 8K HDMI Fibra Optica 10M, Cable HDMI ...",,"{'Marca': 'FeizLink', 'Fabricante': 'FeizLink'...","4,2",311 valoraciones


In [134]:
caracteristicas.pop()

{'Fabricante': 'GSC',
 'Identificador de producto del fabricante': '2600999',
 'Dimensiones del paquete': '17 x 16 x 1,5 cm; 20 gramos',
 'Número de modelo del producto': '2600999',
 'Color': 'Multicolor',
 'Fuente de alimentación': 'CA',
 'Número de productos': '1',
 'Componentes incluidos': 'Evila - Cable conexion hdmi-hdmi negro 1.4/1.8m',
 'Incluye baterías': 'No',
 'Necesita baterías': 'No',
 'Peso del producto': '20 g'}

In [131]:
print(len(nombre))
print(len(precios))
print(len(caracteristicas))
print(len(stars))
print(len(ratings))


58
58
58
57
57
